In [ ]:
import rich
from rich import inspect, pretty
from rich import print
pretty.install()
%load_ext rich

import gc

import multiprocessing as mp
from collections import defaultdict
from functools import partial

import sklearn

import math
import os
import csv
import time
import json
import re
import warnings
import datetime
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
from scipy import stats

from collections import defaultdict, Counter
import itertools

from string import printable

from scipy.spatial.distance import cosine

from scipy.stats import shapiro, ttest_ind, mannwhitneyu
from scipy.spatial import cKDTree

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

import networkx as nx

In [ ]:
from sentence_transformers import SentenceTransformer

## Data loading

In [ ]:
russia_df = pd.read_csv('data/ru_sample_ex.csv')
italy_df = pd.read_csv('data/it_sample_ex.csv')

In [ ]:
italy_docs = [dd for d in italy_df['content'].values for dd in d.split('\n\n')]
russia_docs = [dd for d in russia_df['content'].values for dd in d.split('\n\n')]

In [94]:
italy_sents = [dd for d in italy_df['content'].values for dd in d.split('.') if len(dd) > 20]
russia_sents = [dd for d in russia_df['content'].values for dd in d.split('.') if len(dd) > 20]

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
embeddings_rus = model.encode(russia_docs)
embeddings_ita = model.encode(italy_docs)

In [95]:
sembeddings_rus = model.encode(russia_sents)
sembeddings_ita = model.encode(italy_sents)

In [98]:
S_train, S_test, _, _ = sklearn.model_selection.train_test_split(russia_sents + italy_sents, [1] * len(russia_sents) + [0] * len(italy_sents), test_size=0.5, random_state=42)

In [99]:
Z_train, Z_test, z_train, z_test = sklearn.model_selection.train_test_split(np.vstack([sembeddings_rus, sembeddings_ita]), [1] * len(russia_sents) + [0] * len(italy_sents), test_size=0.5, random_state=42)

In [ ]:
D_train, D_test, _, _ = sklearn.model_selection.train_test_split(russia_docs + italy_docs, [1] * len(russia_docs) + [0] * len(italy_docs), test_size=0.5, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(np.vstack([embeddings_rus, embeddings_ita]), [1] * len(russia_docs) + [0] * len(italy_docs), test_size=0.5, random_state=42)

## paragraph classification

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
outputs = neigh.predict(X_test)
f1_5nn = sklearn.metrics.f1_score(y_test, outputs)
f1_5nn

In [ ]:
neigh1 = KNeighborsClassifier(n_neighbors=1)
neigh1.fit(X_train, y_train)
outputs1 = neigh1.predict(X_test)
f1_1nn = sklearn.metrics.f1_score(y_test, outputs1)
f1_1nn

In [ ]:
neigh3 = KNeighborsClassifier(n_neighbors=3)
outputs3 = neigh3.predict(X_test)
f1_3nn = sklearn.metrics.f1_score(y_test, outputs3)
f1_3nn

In [ ]:
f1_5nn = sklearn.metrics.f1_score(y_test, outputs)
f1_5nn

In [91]:
len(y_test)

4502

In [93]:
sum([outputs[i] != y_test[i] for i in range(len(D_test))])

935

## Error analysis

In [90]:
for i, d in enumerate(D_test):
    if outputs[i] != y_test[i]:
        print(d)
        print('Predicted:', "Russie" if outputs[i] else "Pas Russie")

M. Léon Noël. - Certainement, vous êtes juge.

Predicted: Russie

14. - L'enquête d'utilité publique sur la centrale nucléaire de Plogoff s'achève après quarante-cinq jours marqués 
par des incidents violents avec les forces de l'ordre. Les 24 et 25 mai, 100 000 personnes participent à une fête 
antinucléaire autour du site retenu.

Predicted: Russie

- Non. En réalité, l'un des participants a qualifié les expédients de "remèdes de charlatan". L'expérience prouve 
qu'ils ne font qu'aggraver la situation. Tous les participants réalisent qu'ils ont à affronter des changements 
sociaux pour contrôler des dépenses excessives... Nous avons travaillé à fond sur chaque sujet, sans l'abandonner 
quand on constatait des divergences... Il n'y a eu ni vainqueurs ni vaincus.

...

Predicted: Pas Russie

La contradiction principale que révèlent ces économies tient à ce que, bien souvent, les " calculs centralisés " 
font obstacle à une domination effective des travailleurs sur la transformation des conditions de la production. 
Cet élément de contradiction pèsera désormais plus lourd à mesure que l'économie socialiste se développera et que 
croîtra la tension entre les niveaux d'aspiration des élites et ceux des masses.

Predicted: Pas Russie

La France avait demandé, après le drame de Nice, que le thème du terrorisme soit inscrit à l’agenda du conseil des 
ministres européens des affaires étrangères, lundi 18 juillet, à Bruxelles. Au-delà du « message fort » de tous les
Etats membres – entendu après chaque attentat et même appuyé, cette fois, par le secrétaire d’Etat américain John 
Kerry, présent à la réunion –, Paris sollicitait l’article 42.7 de l’Union européenne. Il énonce que lorsqu’un Etat
membre est l’objet d’une agression armée, tous les autres doivent lui témoigner de la solidarité et lui fournir 
aide et assistance « par tous les moyens nécessaires ».

Predicted: Russie

In [101]:
len(z_train)

12728

## Sentence classification

In [107]:
neigh_sent = KNeighborsClassifier(n_neighbors=5)
neigh_sent.fit(Z_train, z_train)
outputs_sent = neigh_sent.predict(Z_test)
fsent_5nn = sklearn.metrics.f1_score(z_test, outputs_sent)
fsent_5nn

0.7267026396383258

In [108]:
neigh_sent = KNeighborsClassifier(n_neighbors=3)
neigh_sent.fit(Z_train, z_train)
outputs_sent = neigh_sent.predict(Z_test)
fsent_5nn = sklearn.metrics.f1_score(z_test, outputs_sent)
fsent_5nn

0.7208188153310104

In [109]:
neigh_sent = KNeighborsClassifier(n_neighbors=9)
neigh_sent.fit(Z_train, z_train)
outputs_sent = neigh_sent.predict(Z_test)
fsent_5nn = sklearn.metrics.f1_score(z_test, outputs_sent)
fsent_5nn

0.7331925122004516